In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy import linalg as LA
import os

In [2]:
def loadData():
    pb1_file = '../data/pb/hw11pb1.csv'
    pb2_file = '../data/pb/hw11pb2.csv'
    
    pb1 = np.loadtxt(pb1_file, delimiter=',')
    pb2 = np.loadtxt(pb2_file, delimiter=',')
    
    return pb1, pb2

In [3]:
# add log probabilities
def viterbi(obs, states, start_p, trans_p, emit_p):
    V = [{}]
    for st in states:
        V[0][st] = {"prob": start_p[st] * emit_p[st][obs[0]], "prev": None}
    # Run Viterbi when t > 0
    for t in range(1, len(obs)):
        V.append({})
        for st in states:
            max_tr_prob = V[t-1][states[0]]["prob"]*trans_p[states[0]][st]
            prev_st_selected = states[0]
            for prev_st in states[1:]:
                tr_prob = V[t-1][prev_st]["prob"]*trans_p[prev_st][st]
                if tr_prob > max_tr_prob:
                    max_tr_prob = tr_prob
                    prev_st_selected = prev_st
                    
            max_prob = max_tr_prob * emit_p[st][obs[t]]
            V[t][st] = {"prob": max_prob, "prev": prev_st_selected}

    opt = []
    # The highest probability
    max_prob = max(value["prob"] for value in V[-1].values())
    previous = None
    # Get most probable state and its backtrack
    for st, data in V[-1].items():
        if data["prob"] == max_prob:
            opt.append(st)
            previous = st
            break
    # Follow the backtrack till the first observation
    for t in range(len(V) - 2, -1, -1):
        opt.insert(0, V[t + 1][previous]["prev"])
        previous = V[t + 1][previous]["prev"]

    print(' '.join(opt) )

def dptable(V):
    # Print a table of steps from dictionary
    yield " ".join(("%12d" % i) for i in range(len(V)))
    for state in V[0]:
        yield "%.7s: " % state + " ".join("%.7s" % ("%f" % v[state]["prob"]) for v in V)

# Part 1

# a)

References: https://en.wikipedia.org/wiki/Viterbi_algorithm

In [4]:
p1 = 1.0/10.0
p2 = 1.0/2.0
p3 = 1.0/6.0

init_F = p2
init_L = p2

fair = [p3, p3, p3, p3, p3, p3]
loaded = [p1, p1, p1, p1, p1, p2]

pb1, pb2 = loadData()

obs = pb1 #('normal', 'cold', 'dizzy')
states = ('1', '2')
start_p = {'1': 0.5, '2': 0.5}
trans_p = {
   '1' : {'1': 0.95, '2': 0.05},
   '2' : {'1': 0.05, '2': 0.95}
   }
emit_p = {
   '1' : {1.: p3, 2.: p3, 3.: p3, 4.: p3, 5.: p3, 6.: p3},
   '2' : {1.: p1, 2.: p1, 3.: p1, 4.: p1, 5.: p1, 6.: p2}
   }

In [5]:
viterbi(obs,
        states,
        start_p,
        trans_p,
        emit_p)

1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2


# b) 
Refrences: https://en.wikipedia.org/wiki/Forward–backward_algorithm

In [6]:
def fwd_bkw(observations, states, start_prob, trans_prob, emm_prob, end_st):
    # forward part of the algorithm
    fwd = []
    miu = []
    f_prev = {}
    for i, observation_i in enumerate(observations):
        f_curr = {}
        for st in states:
            if i == 0:
                # base case for the forward part
                prev_f_sum = start_prob[st]
            else:
                prev_f_sum = sum(f_prev[k]*trans_prob[k][st] for k in states)

            f_curr[st] = emm_prob[st][observation_i] * prev_f_sum
            
            #if i == 114:
            #    print('alpha_{k}_115 = '.format(k = st), f_curr[st])
        
        miu.append(1/(f_curr['1'] + f_curr['2']))
        
        for st in states:
            f_curr[st] *= miu[i]
        
        fwd.append(f_curr)
        f_prev = f_curr

    #p_fwd = sum(f_curr[k] for k in states)

    # backward part of the algorithm
    bkw = []
    b_prev = {}
    for i, observation_i_plus in enumerate(reversed(observations[1:]+(None,))):
        b_curr = {}
        for st in states:
            if i == 0:
                # base case for backward part
                b_curr[st] = miu[len(miu)-1] * .1 #trans_prob[st][end_st]
            else:
                b_curr[st] = sum(trans_prob[st][l] * emm_prob[l][observation_i_plus] * b_prev[l] for l in states)
           
        if i != 0:
            for st in states:
                b_curr[st] *= miu[len(miu)-i-2]        
                
        bkw.append(b_curr)
        b_prev = b_curr

    #p_bkw = sum(start_prob[l] * emm_prob[l][observations[0]] * b_curr[l] for l in states)

    # merging the two parts

    return fwd, bkw   #, posterior

In [7]:
p1 = 1.0/10.0
p2 = 1.0/2.0
p3 = 1.0/6.0

init_F = p2
init_L = p2

fair = [p3, p3, p3, p3, p3, p3]
loaded = [p1, p1, p1, p1, p1, p2]

pb1, pb2 = loadData()

obs = tuple(pb1)
states = ('1', '2')
end_state = '2'

start_p = {'1': 0.5, '2': 0.5}
trans_p = {
   '1' : {'1': 0.95, '2': 0.05},
   '2' : {'1': 0.05, '2': 0.95}
   }
emit_p = {
   '1' : {1.: p3, 2.: p3, 3.: p3, 4.: p3, 5.: p3, 6.: p3},
   '2' : {1.: p1, 2.: p1, 3.: p1, 4.: p1, 5.: p1, 6.: p2}
   }

In [8]:
t = fwd_bkw(obs,
        states,
        start_p,
        trans_p,
        emit_p, 
        end_state)

print(t[0][115], t[1][115])

{'1': 0.24137669730725378, '2': 0.7586233026927464} {'1': 0.4330503843609442, '2': 0.5086857770067409}


# Part 2

In [28]:
def baum_welch(n_iter=10):
    for n in range(n_iter):
        #alpha = forward(V, a, b, initial_distribution)
        #beta = backward(V, a, b)
        
        alpha, beta = fwd_bkw(obs,
                                states,
                                start_p,
                                trans_p,
                                emit_p, 
                                end_state)
        gammas = {'1':[], '2':[]}
        Eps = {'1':{'1':[], '2':[]}, '2':{'1':[], '2':[]} }
       
        
        for t in range(len(obs)-1):
            g = {}
            E_1 = {}
            E_2 = {}
            
            num1 = alpha[t]['1']*beta[t]['1']
            num2 = alpha[t]['2']*beta[t]['2']
            denom = alpha[t]['1']*beta[t]['1'] + alpha[t]['2']*beta[t]['2']
            
            gammas['1'].append(num1/denom)
            gammas['2'].append(num2/denom)
            
            E_1['1']  = alpha[t]['1'] * trans_p['1']['1']*beta[t+1]['1'] * emit_p['1'][obs[t+1]]
            E_1['2']  = alpha[t]['1'] * trans_p['1']['2']*beta[t+1]['2'] * emit_p['2'][obs[t+1]]
        
            E_2['1']  = alpha[t]['2'] * trans_p['2']['1']*beta[t+1]['1'] * emit_p['1'][obs[t+1]]
            E_2['2']  = alpha[t]['2'] * trans_p['2']['2']*beta[t+1]['2'] * emit_p['2'][obs[t+1]]
            
            denom = E_1['1'] + E_1['2'] + E_2['1'] +E_2['2']
                        
            for st in states:
                E_1[st] = E_1[st] / denom
                E_2[st] = E_2[st] / denom
            
                Eps['1'][st].append(E_1[st])
                Eps['2'][st].append(E_2[st])
                
            
        for st in states:
            start_p[st] = gammas[st][0]
        
            trans_p['1'][st] = sum(Eps['1'][st]) / sum(gammas['1'])
            trans_p['2'][st] = sum(Eps['2'][st]) / sum(gammas['2']) 
            
            V = np.array(obs)
            
            for d in range(1, 7):
                ind = np.where(V == d)[0]
                gammas[st] = np.array(gammas[st])
                emit_p[st][d] = sum(V[np.where(V == d)] * gammas[st][np.where(V == d)-1])
        
        print(n)
        print(start_p)
        print(trans_p)
        print(emit_p)
            
        
        #print(len(gammas))        


In [29]:
p1 = 1.0/10.0
p2 = 1.0/2.0
p3 = 1.0/6.0

init_F = p2
init_L = p2

fair = [p3, p3, p3, p3, p3, p3]
loaded = [p1, p1, p1, p1, p1, p2]

pb1, pb2 = loadData()

obs = tuple(pb2)
states = ('1', '2')
end_state = 'E'

start_p = {'1': 0.5, '2': 0.5}
trans_p = {
   '1' : {'1': 0.94, '2': 0.05},
   '2' : {'1': 0.05, '2': 0.94}
   }
emit_p = {
   '1' : {1.: p3, 2.: p3, 3.: p3, 4.: p3, 5.: p3, 6.: p3},
   '2' : {1.: p1, 2.: p1, 3.: p1, 4.: p1, 5.: p1, 6.: p2}
   }

initital = np.array((0.5, 0.5))



In [30]:
baum_welch()

TypeError: unsupported operand type(s) for -: 'tuple' and 'int'

In [ ]:
print("\n=-=-=-= Pi =-=-=-=")
for key, value in start_p.items():
    print("pi_"+str(key)+": ", value)


print("\n=-=-=-= a =-=-=-=")
for key, value in trans_p.items():
    for k, v in value.items():
        print("a_"+str(key)+str(k)+": ", v)
        
print("\n=-=-=-= b =-=-=-=")
for key, value in emit_p.items():
    for k, v in value.items():
        print("b_"+str(key)+", dice value: " + str(int(k))+": ", v)    
    print("")
